In [1]:
# install lxml using conda
# install beautifulsoup4 and html5lib using pip

In [2]:
import numpy as np
import pandas as pd

In [3]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')

In [4]:
len(tables)

1

In [5]:
failures = tables[0]

In [6]:
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [7]:
# compute the number of bank failures by year
close_timestamps = pd.to_datetime(failures['Closing Date'])

In [8]:
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
2013     24
2014     18
2002     11
2015      8
2016      5
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, dtype: int64

PARSING XML with lxml.objectify

In [9]:
# parse file and get a reference to the root node of XML using getroot
from lxml import objectify

In [10]:
path = 'datasets/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

In [11]:
# root.INDICATOR - parent
# getchildren() - scans tags in the INDICATOR root
# child.tag - select individual tag
# child.pyval - value of selected tag

data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ', 'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [12]:
# convert list of dicts into DF

perf = pd.DataFrame(data)

perf.head()

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95


In [15]:
# extract tag's metadata from HTML snippet
from io import StringIO

In [21]:
tag = '<a href="https://www.google.com">Google</a>'
tag2 = '<h1>This is Python for Data Science</h1>'
root = objectify.parse(StringIO(tag)).getroot()
root2 = objectify.parse(StringIO(tag2)).getroot()

In [22]:
root

<Element a at 0x108424f08>

In [23]:
root.get('href')

'https://www.google.com'

In [24]:
root.text

'Google'

In [25]:
root2

<Element h1 at 0x1060d5648>

In [27]:
root2.text

'This is Python for Data Science'

BINARY DATA FORMATS

In [28]:
# writing data into pickle format
frame = pd.read_csv('examples/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [29]:
frame.to_pickle('examples/frame_pickle')

In [30]:
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


USING HDF5 FORMAT

In [31]:
# can store large quantities of scientific data
# available as a C library with interfaces to other languages
# HDF5 - hierarchical data format
# supports on fly compression with modes
# HDFStore class - works like a dictionary able to handle low-level details

In [33]:
frame = pd.DataFrame({'a': np.random.randn(100)})

store = pd.HDFStore('mydata.h5')

store['obj1'] = frame

store['obj1_col'] = frame['a']

store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [36]:
store['obj1'].head()

,a
0,-0.119035
1,-0.948321
2,-0.460960
3,0.101210
4,1.240919


In [37]:
# HDFStore - storage schemas - 'fixed' and 'table'
# table is slower but supports query ops
store.put('obj2', frame, format='table')

store.select('obj2', where=['index >= 10 and index <= 15'])

,a
10,0.138589
11,-0.389294
12,-1.883468
13,0.498393
14,0.181005
15,-1.009128


In [38]:
store.close()

In [39]:
# shortcut - pandas.read_hdf
frame.to_hdf('mydata.h5', 'obj3', format='table')

pd.read_hdf('mydata.h5', 'obj3', where=['index < 5'])

,a
0,-0.119035
1,-0.948321
2,-0.460960
3,0.101210
4,1.240919


READING EXCEL FILES

In [41]:
# using ExcelFile
xlsx = pd.ExcelFile('examples/ex1.xlsx')

# useful to define multiple sheets at a time
pd.read_excel(xlsx, 'Sheet1')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [42]:
# can pass filename to pandas.read_excel
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [43]:
# write pandas data to Excel format - create ExcelWriter
writer = pd.ExcelWriter('examples/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

In [44]:
# can pass a file path to to_excel and avoid ExcelWriter
frame.to_excel('examples/ex2.xlsx')

In [45]:
# read new file
new_file = pd.read_excel('examples/ex2.xlsx', 'Sheet1')
new_file

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


INTERACTING WITH WEB APIs

In [46]:
# websites have public APIs - feeds - JSON format
# find last 30 GitHub issues for pandas, make a GET HTTP request
import requests

In [47]:
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'

In [48]:
resp = requests.get(url)
resp

<Response [200]>

In [83]:
# response object's json method will return a dictionary
# with JSON parsed into native Python objects

In [99]:
for x in data:
    for key in x:
        print(key)

url
repository_url
labels_url
comments_url
events_url
html_url
id
node_id
number
title
user
labels
state
locked
assignee
assignees
milestone
comments
created_at
updated_at
closed_at
author_association
pull_request
body
url
repository_url
labels_url
comments_url
events_url
html_url
id
node_id
number
title
user
labels
state
locked
assignee
assignees
milestone
comments
created_at
updated_at
closed_at
author_association
body
url
repository_url
labels_url
comments_url
events_url
html_url
id
node_id
number
title
user
labels
state
locked
assignee
assignees
milestone
comments
created_at
updated_at
closed_at
author_association
body
url
repository_url
labels_url
comments_url
events_url
html_url
id
node_id
number
title
user
labels
state
locked
assignee
assignees
milestone
comments
created_at
updated_at
closed_at
author_association
body
url
repository_url
labels_url
comments_url
events_url
html_url
id
node_id
number
title
user
labels
state
locked
assignee
assignees
milestone
comments
created_at
up

In [91]:
data = resp.json()

print("User: ", data[0]['user']['login'])
print("Title: ", data[0]['title'])

User:  diogo-bastos
Title:  DOC: Fix quotes position in Timestamp and Timedelta


In [93]:
# passing data in DF and extract fields of interest
issues = pd.DataFrame(data, columns=['number', 'title', 'labels', 'state'])
issues.head()

,number,title,labels,state
0,24243,DOC: Fix quotes position in Timestamp and Time...,[],open
1,24242,TST: Hypothesis may draw a date outside of dat...,[],open
2,24241,Series.apply on categorical with NaN has wrong...,[],open
3,24240,DOC: Fix flake8 issues in doc/source/whatsnew/...,"[{'id': 211029535, 'node_id': 'MDU6TGFiZWwyMTE...",open
4,24239,DOC: Fix flake8 issues in doc/source/whatsnew...,"[{'id': 211029535, 'node_id': 'MDU6TGFiZWwyMTE...",open


In [113]:
# another example
user_details = pd.DataFrame(data, columns=["user", 'title'])
user_details.head()

,user,title
0,"{'login': 'diogo-bastos', 'id': 11160226, 'nod...",DOC: Fix quotes position in Timestamp and Time...
1,"{'login': 'topper-123', 'id': 26364415, 'node_...",TST: Hypothesis may draw a date outside of dat...
2,"{'login': 'tchklovski', 'id': 278558, 'node_id...",Series.apply on categorical with NaN has wrong...
3,"{'login': 'saurav2608', 'id': 4477229, 'node_i...",DOC: Fix flake8 issues in doc/source/whatsnew/...
4,"{'login': 'saurav2608', 'id': 4477229, 'node_i...",DOC: Fix flake8 issues in doc/source/whatsnew...


INTERACTING WITH DATABASES

In [114]:
# creating SQLite db using sqlite3 driver
import sqlite3

In [116]:
query = "CREATE TABLE test (a VARCHAR(20), b VARCHAR(20), c REAL, d INTEGER);"

In [117]:
con = sqlite3.connect('mydata.sqlite')

In [118]:
con.execute(query)

In [119]:
con.commit()

In [120]:
# insert new data
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]

In [121]:
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

In [122]:
con.executemany(stmt, data)

In [123]:
con.commit()

In [124]:
# selection of data returns a list of tuples
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [125]:
# passing list of tuples to DF constructor
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [126]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [127]:
# using SQLAlchemy to read data from above DF
import sqlalchemy as sqla

In [129]:
db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
